In [33]:
# Change to project's root directory
%cd "/content/drive/MyDrive/Colab Notebooks/FinancialForecastingDashboard"

/content/drive/MyDrive/Colab Notebooks/FinancialForecastingDashboard


In [34]:
import os
from google.colab import userdata

# Get username and password from Colab secrets
db_user = userdata.get('DB_USER')
db_pass = userdata.get('DB_PASS')

# Install PostgreSQL
!apt-get -y -qq install postgresql-14 >/dev/null

# Start the PostgreSQL server
!sudo service postgresql start

# Install dbt and its postgres adapter
!pip install dbt-postgres

 * Starting PostgreSQL 14 database server
   ...done.


In [35]:
# Get the rest of the environment variables from Colab secrets.
db_host = userdata.get('DB_HOST')
db_port = userdata.get('DB_PORT')
db_name = userdata.get('DB_NAME')

!DB_HOST={db_host} DB_PORT={db_port} DB_NAME={db_name} DB_USER={db_user} DB_PASS={db_pass} python "scripts/database_setup.py"

Successfully connected to the database.
Table 'raw_saas_metrics' is ready.
Successfully ingested 5000 records into raw_saas_metrics.
Database connection closed.


In [36]:
from google.colab import userdata
import os

# Create the .dbt directory, if it doesn't exist.
dbt_dir = os.path.join(os.path.expanduser("~"), ".dbt")
if not os.path.exists(dbt_dir):
  os.makedirs(dbt_dir)

# Define the content of profiles.yml file.
profiles_content = f"""
financial_forecasting:
  target: dev
  outputs:
    dev:
      type: postgres
      host: {userdata.get('DB_HOST')}
      port: {userdata.get('DB_PORT')}
      user: {userdata.get('DB_USER')}
      password: {userdata.get('DB_PASS')}
      dbname: {userdata.get('DB_NAME')}
      schema: public
"""

# Write the content to the profiles.yml file
profiles_path = os.path.join(dbt_dir, "profiles.yml")
with open(profiles_path, "w") as f:
  f.write(profiles_content)
print("dbt profiles.yml file has been created successfully.")

dbt profiles.yml file has been created successfully.


In [37]:
# Change to dbt project's directory
%cd "/content/drive/MyDrive/Colab Notebooks/FinancialForecastingDashboard/financial_forecasting"

/content/drive/MyDrive/Colab Notebooks/FinancialForecastingDashboard/financial_forecasting


In [38]:
# Verify the setup
!dbt debug --profiles-dir ~/.dbt --project-dir .

07:15:50  Running with dbt=1.10.11
07:15:50  dbt version: 1.10.11
07:15:50  python version: 3.12.11
07:15:50  python path: /usr/bin/python3
07:15:50  os info: Linux-6.6.97+-x86_64-with-glibc2.35
07:15:50  Using profiles dir at /root/.dbt
07:15:50  Using profiles.yml file at /root/.dbt/profiles.yml
07:15:50  Using dbt_project.yml file at ./dbt_project.yml
07:15:50  adapter type: postgres
07:15:50  adapter version: 1.9.1
07:15:51  Configuration:
07:15:51    profiles.yml file [OK found and valid]
07:15:51    dbt_project.yml file [OK found and valid]
07:15:51  Required dependencies:
07:15:51   - git [OK found]

07:15:51  Connection:
07:15:51    host: localhost
07:15:51    port: 5432
07:15:51    user: root
07:15:51    database: financial_data
07:15:51    schema: public
07:15:51    connect_timeout: 10
07:15:51    role: None
07:15:51    search_path: None
07:15:51    keepalives_idle: 0
07:15:51    sslmode: None
07:15:51    sslcert: None
07:15:51    sslkey: None
07:15:51    sslrootcert: None
07

In [39]:
!mkdir -p "/content/drive/MyDrive/Colab Notebooks/FinancialForecastingDashboard/financial_forecasting/models/marts"

In [40]:
%%writefile "/content/drive/MyDrive/Colab Notebooks/FinancialForecastingDashboard/financial_forecasting/models/marts/fact_monthly_revenue.sql"

-- Build the final table for analysis.
-- Adding the MRR metric to data from the staging layer.
-- Uses a dbt ref macro.

SELECT
    DATE_TRUNC('month', subscription_start_date) AS subscription_month,
    SUM(monthly_recurring_revenue) AS monthly_recurring_revenue
FROM
    {{ ref('stg_raw_saas_metrics_data') }}
GROUP BY 1
ORDER BY 1

Overwriting /content/drive/MyDrive/Colab Notebooks/FinancialForecastingDashboard/financial_forecasting/models/marts/fact_monthly_revenue.sql


In [41]:
!dbt parse

07:16:10  Running with dbt=1.10.11
07:16:11  Registered adapter: postgres=1.9.1
07:16:12  [WARNING]: Configuration paths exist in your dbt_project.yml file which do not apply to any resources.
There are 1 unused configuration paths:
- models.financial_forecasting.example
07:16:12  Performance info: /content/drive/MyDrive/Colab Notebooks/FinancialForecastingDashboard/financial_forecasting/target/perf_info.json


In [42]:
# This runs the stg_raw_saas_metrics_data model & the fact_monthly_revenue model.
# Creates the fact_monthly_revenue table in the db, completing the pipeline.

!dbt run

07:16:19  Running with dbt=1.10.11
07:16:20  Registered adapter: postgres=1.9.1
07:16:20  [WARNING]: Configuration paths exist in your dbt_project.yml file which do not apply to any resources.
There are 1 unused configuration paths:
- models.financial_forecasting.example
07:16:20  Found 2 models, 4 data tests, 1 source, 434 macros
07:16:20  
07:16:20  Concurrency: 1 threads (target='dev')
07:16:20  
07:16:20  1 of 2 START sql view model public.stg_raw_saas_metrics_data ................... [RUN]
07:16:21  1 of 2 OK created sql view model public.stg_raw_saas_metrics_data .............. [CREATE VIEW in 0.18s]
07:16:21  2 of 2 START sql view model public.fact_monthly_revenue ........................ [RUN]
07:16:21  2 of 2 OK created sql view model public.fact_monthly_revenue ................... [CREATE VIEW in 0.09s]
07:16:21  
07:16:21  Finished running 2 view models in 0 hours 0 minutes and 0.47 seconds (0.47s).
07:16:21  
07:16:21  Completed successfully
07:16:21  
07:16:21  Done. PASS=